In [17]:
import pandas as pd
import random
from faker import Faker

file_gw = "MakeOrders2.xlsx"
# Initialize Faker
fake = Faker()

# Define number of rows and column structure
num_rows = 500  # Change this to the desired number of rows
columns = [
    {"name": "id_order", "type": "id_order"},
    {"name": "id_employee", "type": "id_employee"},
    {"name": "date_order", "type": "date_order"},
    {"name": "payment_status", "type": "payment_status"},
    {"name": "table_id", "type": "table_id"}  # Adding a column for meja (table ID)
]

# Generate synthetic data
synthetic_data = pd.DataFrame()

# Generate unique `id_order`, `id_employee`, and `table_id`
id_orders = [f"OO{str(i).zfill(4)}" for i in range(1, num_rows + 1)]
id_employees = [f"E{str(i).zfill(4)}" for i in range(1, num_rows + 1)]
table_ids = [f"M{str(i).zfill(2)}" for i in range(1, 21)]  # Table IDs from M01 to M20

for column in columns:
    if column["type"] == "id_order":
        synthetic_data[column["name"]] = id_orders
    elif column["type"] == "id_employee":
        synthetic_data[column["name"]] = id_employees
    elif column["type"] == "date_order":
        # Generate random fake dates in SQL format
        synthetic_data[column["name"]] = [
            fake.date_between(start_date="-5y", end_date="today").strftime("%Y-%m-%d") for _ in range(num_rows)
        ]
    elif column["type"] == "table_id":
        # Randomly assign table IDs
        synthetic_data[column["name"]] = [random.choice(table_ids) for _ in range(num_rows)]
    elif column["type"] == "payment_status":
        synthetic_data[column["name"]] = "Paid"
    else:
        synthetic_data[column["name"]] = [f"Data_{i}" for i in range(1, num_rows + 1)]

# Save the dataset to an Excel file
output_path = file_gw
synthetic_data.to_excel(output_path, index=False)

print(f"Synthetic dataset saved to {output_path}")



Synthetic dataset saved to MakeOrders2.xlsx


In [18]:
import pandas as pd
import mysql.connector

# Define your MariaDB connection
server = 'localhost'  # Replace with your server name
username = 'root'     # Your MariaDB username
password = ''         # Your MariaDB password
database = 'uasdb'    # The database name
table_name = 'orders' # The table name in your database

# Establish connection
try:
    conn = mysql.connector.connect(
        host=server,
        user=username,
        password=password,
        database=database
    )
    cursor = conn.cursor()
    print("Connection to the database was successful.")
except mysql.connector.Error as err:
    print(f"Error: {err}")
    exit()

# Load the dataset
input_file = file_gw
synthetic_data = pd.read_excel(input_file)

# Debugging: Check column names and data
print("Columns in Excel file:", synthetic_data.columns)
print("First few rows:\n", synthetic_data.head())

# Ensure proper column mapping
synthetic_data.rename(columns={"Id_emp": "id_employee", "table_id": "id_meja"}, inplace=True)

# Insert data into MariaDB
try:
    for index, row in synthetic_data.iterrows():
        cursor.execute(f"""
            INSERT INTO {table_name} (id_order, id_employee, id_meja, date_order, payment_status)
            VALUES (%s, %s, %s, %s, %s)
        """, 
        (row['id_order'], row['id_employee'], row['id_meja'], row['date_order'], row['payment_status']))

    conn.commit()
    print("Data successfully inserted into MariaDB!")
except Exception as e:
    print(f"Error inserting data: {e}")
finally:
    # Close connection
    cursor.close()
    conn.close()
    print("Database connection closed.")



Connection to the database was successful.
Columns in Excel file: Index(['id_order', 'id_employee', 'date_order', 'payment_status', 'table_id'], dtype='object')
First few rows:
   id_order id_employee  date_order payment_status table_id
0   OO0001       E0001  2023-01-19           Paid      M18
1   OO0002       E0002  2020-07-21           Paid      M16
2   OO0003       E0003  2023-07-13           Paid      M11
3   OO0004       E0004  2021-05-04           Paid      M11
4   OO0005       E0005  2023-05-10           Paid      M03
Error inserting data: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`uasdb`.`orders`, CONSTRAINT `orders_ibfk_1` FOREIGN KEY (`id_employee`) REFERENCES `employee` (`id_employee`) ON DELETE CASCADE)
Database connection closed.
